In [2]:
import pandas as pd
import numpy as np

# Load the one-minute stock prices CSV file (adjust the path as needed)
df = pd.read_csv('../1min_stock_prices.csv')

# If a Date column exists, parse it as datetime and sort by time
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', inplace=True)
    df.reset_index(drop=True, inplace=True)
    

#############################################
# 1. MACD Calculation
#############################################
# MACD is computed using two EMAs of the closing price.
# Typical periods: 12 and 26 for EMAs and 9 for the signal line.
df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = df['EMA12'] - df['EMA26']
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
df['Hist'] = df['MACD'] - df['Signal']

#############################################
# 2. RSI Calculation (Period = 14)
#############################################
delta = df['Close'].diff()
gain = delta.copy()
loss = delta.copy()
gain[gain < 0] = 0
loss[loss > 0] = 0
loss = loss.abs()

window_length = 14
avg_gain = gain.rolling(window=window_length, min_periods=window_length).mean()
avg_loss = loss.rolling(window=window_length, min_periods=window_length).mean()
rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))

#############################################
# 3. KDJ Calculation (Period = 9)
#############################################
# Compute the Raw Stochastic Value (RSV)
period = 9
df['Low_Min'] = df['Low'].rolling(window=period, min_periods=period).min()
df['High_Max'] = df['High'].rolling(window=period, min_periods=period).max()
df['RSV'] = (df['Close'] - df['Low_Min']) / (df['High_Max'] - df['Low_Min']) * 100

# Initialize K, D, and J. Typically start K and D at 50.
df['K'] = np.nan
df['D'] = np.nan
df['J'] = np.nan

for i in range(len(df)):
    if np.isnan(df.loc[i, 'RSV']):
        continue
    if i == df['RSV'].first_valid_index():
        df.loc[i, 'K'] = 50
        df.loc[i, 'D'] = 50
    else:
        df.loc[i, 'K'] = (2/3) * df.loc[i-1, 'K'] + (1/3) * df.loc[i, 'RSV']
        df.loc[i, 'D'] = (2/3) * df.loc[i-1, 'D'] + (1/3) * df.loc[i, 'K']
    df.loc[i, 'J'] = 3 * df.loc[i, 'K'] - 2 * df.loc[i, 'D']

#############################################
# 4. OSC Calculation: Price Oscillator
#############################################
# Using the percentage difference between 5-minute and 10-minute SMAs.
sma_fast = df['Close'].rolling(window=5).mean()
sma_slow = df['Close'].rolling(window=10).mean()
df['OSC'] = (sma_fast - sma_slow) / sma_slow * 100

#############################################
# 5. BOLL Calculation: Bollinger Bands
#############################################
# Bollinger Bands using a 20-period SMA and 2 standard deviations.
boll_period = 20
df['BOLL_Mid'] = df['Close'].rolling(window=boll_period).mean()
df['BOLL_STD'] = df['Close'].rolling(window=boll_period).std()
df['BOLL_Upper'] = df['BOLL_Mid'] + 2 * df['BOLL_STD']
df['BOLL_Lower'] = df['BOLL_Mid'] - 2 * df['BOLL_STD']

#############################################
# 6. BIAS Calculation: Price Bias Indicator
#############################################
# BIAS is the percentage difference between the closing price and its 6-period SMA.
bias_period = 6
sma_bias = df['Close'].rolling(window=bias_period).mean()
df['BIAS'] = (df['Close'] - sma_bias) / sma_bias * 100

#############################################
# Display or save results
#############################################
# Print the last few rows to check the calculated indicators
print(df.tail())

# Optionally, save the results to a new CSV file
# df.to_csv('1min_stock_prices_with_indicators.csv', index=False)



                   datetime      Open      High       Low     Close   Volume  \
212194  2025-03-01 00:55:00  293.6900  293.8000  293.5000  293.8000   2423.0   
212195  2025-03-01 00:56:00  293.8100  294.1800  293.8001  294.1365  12443.0   
212196  2025-03-01 00:57:00  294.1365  294.1365  293.9152  294.1187   5124.0   
212197  2025-03-01 00:58:00  294.1200  294.1800  294.0500  294.1507   5991.0   
212198  2025-03-01 00:59:00  294.1600  294.4500  294.1600  294.4000  17177.0   

       Ticker       EMA12       EMA26      MACD  ...        RSV   K   D   J  \
212194   TSLA  293.742630  293.641312  0.101318  ...  76.923077 NaN NaN NaN   
212195   TSLA  293.803225  293.677992  0.125233  ...  93.602941 NaN NaN NaN   
212196   TSLA  293.851760  293.710637  0.141122  ...  90.985294 NaN NaN NaN   
212197   TSLA  293.897751  293.743235  0.154516  ...  95.691176 NaN NaN NaN   
212198   TSLA  293.975020  293.791884  0.183136  ...  94.736842 NaN NaN NaN   

             OSC    BOLL_Mid  BOLL_STD  BOLL